In [1]:
from pyspark.ml.recommendation import ALSModel
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pandas as pd
import os
import sys

In [2]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [3]:
# Dynamic path settings
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
TRAIN_TEST_SPLIT_DIR = os.path.join(BASE_DIR, "train_test_split")

In [4]:
spark = SparkSession.builder \
    .appName("RecommenderSystem") \
    .master("local[*]") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.cores", "4") \
    .config("spark.driver.cores", "4") \
    .config("spark.python.worker.reuse", "false") \
    .config("spark.local.dir", "/tmp/spark-temp") \
    .config("spark.network.timeout", "600s") \
    .config("spark.executor.heartbeatInterval", "60s") \
    .getOrCreate()

In [5]:
# Load test data
test_sales_path = os.path.join(TRAIN_TEST_SPLIT_DIR, "test_sales_data.csv")
test_df = pd.read_csv(test_sales_path)

In [6]:
# Ensure there are data rows in test data
if test_df.empty:
    raise ValueError("Test DataFrame is empty.")

In [7]:
# Convert interaction_type column to numeric
test_df['interaction_type'] = pd.to_numeric(test_df['interaction_type'], errors='coerce')
test_df.dropna(subset=['interaction_type'], inplace=True)

In [8]:
# Check if the dataframe is still empty after conversion and dropping NA values
if test_df.empty:
    raise ValueError("Test DataFrame is empty after converting 'interaction_type' to numeric and dropping NA values.")

In [9]:
# Convert test data to Spark DataFrame
test_data = [Row(user_id=int(row.user_id), product_id=int(row.product_id), interaction_type=float(row['interaction_type'])) for index, row in test_df.iterrows()]
test_df_spark = spark.createDataFrame(test_data)

In [10]:
# Load the trained model using Spark's load method
model_path = os.path.join(BASE_DIR, "als_model")
model = ALSModel.load(model_path)

In [11]:
# Making predictions on test data
predictions = model.transform(test_df_spark)

In [14]:
predictions.show()

+-------+----------+----------------+----------+
|user_id|product_id|interaction_type|prediction|
+-------+----------+----------------+----------+
+-------+----------+----------------+----------+



In [12]:
# Evaluation metrics
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="interaction_type", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print(f"Root-mean-square error = {rmse}")

IllegalArgumentException: requirement failed: Nothing has been added to this summarizer.

In [ ]:
# Getting recommendations for users
user_recs = model.recommendForAllUsers(10)
user_recs.show()